In [58]:
import pandas as pd

df = (pd.read_csv('./data/naut_all.csv')
      .assign(sents=lambda x: x.sents.apply(eval))
      )

In [105]:
df.iloc[0].content

'b\'\\n  <h1>Editor\\\'s Note</h1>\\n\\n  <h4 class="foc"/>\\n\\n  <h2>The Story of Nautilus</h2>\\n\\n  <h3>The nautilus, our namesake, is the subject of science, math and myth.</h3>\\n\\n  <h4>By Michael Segal<br/></h4>\\n\\n  <div class="diamond">\\n    \\xe2\\x99\\xa6\\n  </div>\\n\\n  <div class="cont">\\n    <p><b>Behold the humble nautilus.</b> Just about a foot in diameter, it is a slow bottom-dweller with short tentacles that moves through the water with an unsteady wobble. It\\xe2\\x80\\x99s also 500 million years old and, in its day, was the best and brightest, using its newly evolved depth control to lay waste to acre after acre of scuttling crustacean prey.</p>\\n\\n    <p>We became interested in it here at <i>Nautilus</i> because, well, we stole its name. But also because (for a mollusk) it represents a remarkable intersection of science, math, myth, and culture. Since that is exactly the kind of intersection we love to write about, we decided to put together a little \\x

In [173]:
extract_article_metadata(df.iloc[12].content)

[] ['Dress for Evolutionary Success'] ['What makes us quintessentially human? Fashion!'] ['Culture | Anthropology', 'By Jeanne Carstensen\n  Illustrations by John Hendrix']


{'headline': 'Dress for Evolutionary Success',
 'byline': 'What makes us quintessentially human? Fashion!',
 'segment': 'Culture | Anthropology',
 'author': 'By Jeanne Carstensen\n  Illustrations by John Hendrix'}